In [3]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from natsort import natsorted
from ultralytics import YOLO


## 이미지를 영상으로 변환

In [4]:
def images_to_video(
    img_dir: str,
    out_video_path: str,
    fps: float = 7.5,
    img_exts=(".jpg", ".png")
):
    """
    img_dir 안의 이미지들을 자연 정렬 후 7.5fps 영상으로 저장.
    """
    img_paths = []
    for ext in img_exts:
        img_paths.extend(glob.glob(os.path.join(img_dir, f"*{ext}")))
    img_paths = natsorted(img_paths)

    if not img_paths:
        raise RuntimeError(f"[ERROR] No images found in {img_dir}")

    # 첫 프레임으로 크기 획득
    sample = cv2.imread(img_paths[0])
    if sample is None:
        raise RuntimeError(f"[ERROR] Failed to read first image: {img_paths[0]}")
    h, w = sample.shape[:2]

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_video_path, fourcc, fps, (w, h))

    for p in img_paths:
        frame = cv2.imread(p)
        if frame is None:
            print(f"[WARN] skip invalid image: {p}")
            continue
        writer.write(frame)

    writer.release()
    print(f"[INFO] Video saved: {out_video_path}")


In [26]:
# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"demo\origin\video_normal_new_002.mp4"
OUT_OC_FE_DIR1 = r"test_data\test_video\video_normal_new_002(fast)"
images_to_video(OUT_OC_FE_DIR1, OUT_YOLO_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"demo\origin\video_normal_new_003.mp4"
OUT_OC_FE_DIR2 = r"test_data\test_video\video_normal_new_003"
images_to_video(OUT_OC_FE_DIR2, OUT_YOLO_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"demo\origin\video_missing1_new_002.mp4"
OUT_OC_FE_DIR3 = r"test_data\test_video\video_missing1_new_002"
images_to_video(OUT_OC_FE_DIR3, OUT_YOLO_VIDEO, fps=7.5)

[INFO] Video saved: demo\origin\video_normal_new_002.mp4
[INFO] Video saved: demo\origin\video_normal_new_003.mp4
[INFO] Video saved: demo\origin\video_missing1_new_002.mp4


## yolo 시각화

In [5]:
IMG_DIR1    = r"test_data/test_video/video_normal_new_002(fast)"      # 시각화할 이미지들이 들어있는 폴더
IMG_DIR2    = r"test_data\test_video\video_normal_new_003"
IMG_DIR3    = r"test_data\test_video\video_missing1_new_002"

In [7]:
# ===== 경로 설정 =====
WEIGHT_OC = r"G:\GitProjects\sessac_project\yolo\best_openclose.pt"   # open/close 모델 가중치 경로
WEIGHT_FE = r"G:\GitProjects\sessac_project\yolo\best_fullempty.pt"   # full/empty 모델 가중치 경로

OUT_OC_FE_DIR = r"demo\yolo"        # 시각화 이미지 저장 폴더
os.makedirs(OUT_OC_FE_DIR, exist_ok=True)

# ===== 모델 로드 =====
model_oc = YOLO(WEIGHT_OC)
model_fe = YOLO(WEIGHT_FE)

In [9]:
# ===== 샘플 이미지 목록 =====
sample_imgs = []
for ext in ("*.jpg", "*.png"):
    sample_imgs.extend(glob.glob(os.path.join(IMG_DIR1, ext)))
sample_imgs = natsorted(sample_imgs)

print(f"[INFO] Found {len(sample_imgs)} images.")

# 클래스 이름 (학습할 때 쓴 순서와 맞춰야 함)
OC_NAMES = ["open", "close"]   # open/close 모델
FE_NAMES = ["empty", "full"]   # full/empty 모델

for img_path in sample_imgs:
    # 이미지 로드 (BGR -> RGB)
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"[WARN] 이미지 로드 실패: {img_path}")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # -------- 1) 두 모델로 각각 추론 --------
    res_oc = model_oc(img_path, imgsz=640, conf=0.25)[0]
    res_fe = model_fe(img_path, imgsz=640, conf=0.25)[0]

    # 시각화용 복사본
    vis = img_rgb.copy()

    # -------- 2) open/close 박스 (얇은 초록색) --------
    if res_oc.boxes is not None and len(res_oc.boxes) > 0:
        for box, cls, conf in zip(res_oc.boxes.xyxy, res_oc.boxes.cls, res_oc.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"OC:{OC_NAMES[cls_idx]} {conf.item():.2f}"

            # 얇은 초록색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(0, 255, 0),   # BGR in cv2, but vis is RGB라 상관 없음 (색만 맞추면 됨)
                thickness=2
            )
            # 라벨 텍스트 (박스 위에)
            cv2.putText(
                vis,
                label,
                (x1, max(y1 - 5, 0)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 3) full/empty 박스 (두꺼운 빨간색) --------
    if res_fe.boxes is not None and len(res_fe.boxes) > 0:
        for box, cls, conf in zip(res_fe.boxes.xyxy, res_fe.boxes.cls, res_fe.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"FE:{FE_NAMES[cls_idx]} {conf.item():.2f}"

            # 더 두꺼운 빨간색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(255, 0, 0),   # BGR
                thickness=4
            )
            # 라벨 텍스트 (박스 아래쪽에)
            cv2.putText(
                vis,
                label,
                (x1, min(y2 + 20, vis.shape[0] - 5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 4) 시각화 결과 저장 + 필요 시 화면에도 표시 --------
    out_name = os.path.basename(img_path)
    out_path = os.path.join(OUT_OC_FE_DIR, out_name)

    # vis는 RGB라 BGR로 변환 후 저장
    vis_bgr = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, vis_bgr)
    print(f"[INFO] Saved vis image: {out_path}")

    # 필요하면 바로 plt로 보여주기
    # plt.figure(figsize=(6, 6))
    # plt.title(os.path.basename(img_path))
    # plt.imshow(vis)
    # plt.axis('off')
    # plt.show()


[INFO] Found 235 images.



image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_002(fast)\frame_000000.jpg: 384x640 (no detections), 92.1ms
Speed: 1.4ms preprocess, 92.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_002(fast)\frame_000000.jpg: 384x640 1 full, 66.0ms
Speed: 1.1ms preprocess, 66.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Saved vis image: demo\yolo\frame_000000.jpg

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_002(fast)\frame_000001.jpg: 384x640 (no detections), 70.5ms
Speed: 1.2ms preprocess, 70.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_002(fast)\frame_000001.jpg: 384x640 1 full, 52.5ms
Speed: 1.2ms preprocess, 52.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Saved vis image: 

In [10]:
# ===== 샘플 이미지 목록 =====
sample_imgs = []
for ext in ("*.jpg", "*.png"):
    sample_imgs.extend(glob.glob(os.path.join(IMG_DIR2, ext)))
sample_imgs = natsorted(sample_imgs)

print(f"[INFO] Found {len(sample_imgs)} images.")

# 클래스 이름 (학습할 때 쓴 순서와 맞춰야 함)
OC_NAMES = ["open", "close"]   # open/close 모델
FE_NAMES = ["empty", "full"]   # full/empty 모델

for img_path in sample_imgs:
    # 이미지 로드 (BGR -> RGB)
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"[WARN] 이미지 로드 실패: {img_path}")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # -------- 1) 두 모델로 각각 추론 --------
    res_oc = model_oc(img_path, imgsz=640, conf=0.25)[0]
    res_fe = model_fe(img_path, imgsz=640, conf=0.25)[0]

    # 시각화용 복사본
    vis = img_rgb.copy()

    # -------- 2) open/close 박스 (얇은 초록색) --------
    if res_oc.boxes is not None and len(res_oc.boxes) > 0:
        for box, cls, conf in zip(res_oc.boxes.xyxy, res_oc.boxes.cls, res_oc.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"OC:{OC_NAMES[cls_idx]} {conf.item():.2f}"

            # 얇은 초록색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(0, 255, 0),   # BGR in cv2, but vis is RGB라 상관 없음 (색만 맞추면 됨)
                thickness=2
            )
            # 라벨 텍스트 (박스 위에)
            cv2.putText(
                vis,
                label,
                (x1, max(y1 - 5, 0)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 3) full/empty 박스 (두꺼운 빨간색) --------
    if res_fe.boxes is not None and len(res_fe.boxes) > 0:
        for box, cls, conf in zip(res_fe.boxes.xyxy, res_fe.boxes.cls, res_fe.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"FE:{FE_NAMES[cls_idx]} {conf.item():.2f}"

            # 더 두꺼운 빨간색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(255, 0, 0),   # BGR
                thickness=4
            )
            # 라벨 텍스트 (박스 아래쪽에)
            cv2.putText(
                vis,
                label,
                (x1, min(y2 + 20, vis.shape[0] - 5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 4) 시각화 결과 저장 + 필요 시 화면에도 표시 --------
    out_name = os.path.basename(img_path)
    out_path = os.path.join(OUT_OC_FE_DIR, out_name)

    # vis는 RGB라 BGR로 변환 후 저장
    vis_bgr = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, vis_bgr)
    print(f"[INFO] Saved vis image: {out_path}")

    # 필요하면 바로 plt로 보여주기
    # plt.figure(figsize=(6, 6))
    # plt.title(os.path.basename(img_path))
    # plt.imshow(vis)
    # plt.axis('off')
    # plt.show()

[INFO] Found 452 images.

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_003\frame_000000.jpg: 384x640 (no detections), 83.4ms
Speed: 1.3ms preprocess, 83.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_003\frame_000000.jpg: 384x640 (no detections), 62.7ms
Speed: 1.8ms preprocess, 62.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Saved vis image: demo\yolo\frame_000000.jpg

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_003\frame_000001.jpg: 384x640 (no detections), 60.9ms
Speed: 3.3ms preprocess, 60.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_normal_new_003\frame_000001.jpg: 384x640 (no detections), 58.3ms
Speed: 1.1ms preprocess, 58.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
[INF

In [11]:
# ===== 샘플 이미지 목록 =====
sample_imgs = []
for ext in ("*.jpg", "*.png"):
    sample_imgs.extend(glob.glob(os.path.join(IMG_DIR3, ext)))
sample_imgs = natsorted(sample_imgs)

print(f"[INFO] Found {len(sample_imgs)} images.")

# 클래스 이름 (학습할 때 쓴 순서와 맞춰야 함)
OC_NAMES = ["open", "close"]   # open/close 모델
FE_NAMES = ["empty", "full"]   # full/empty 모델

for img_path in sample_imgs:
    # 이미지 로드 (BGR -> RGB)
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"[WARN] 이미지 로드 실패: {img_path}")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # -------- 1) 두 모델로 각각 추론 --------
    res_oc = model_oc(img_path, imgsz=640, conf=0.25)[0]
    res_fe = model_fe(img_path, imgsz=640, conf=0.25)[0]

    # 시각화용 복사본
    vis = img_rgb.copy()

    # -------- 2) open/close 박스 (얇은 초록색) --------
    if res_oc.boxes is not None and len(res_oc.boxes) > 0:
        for box, cls, conf in zip(res_oc.boxes.xyxy, res_oc.boxes.cls, res_oc.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"OC:{OC_NAMES[cls_idx]} {conf.item():.2f}"

            # 얇은 초록색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(0, 255, 0),   # BGR in cv2, but vis is RGB라 상관 없음 (색만 맞추면 됨)
                thickness=2
            )
            # 라벨 텍스트 (박스 위에)
            cv2.putText(
                vis,
                label,
                (x1, max(y1 - 5, 0)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 3) full/empty 박스 (두꺼운 빨간색) --------
    if res_fe.boxes is not None and len(res_fe.boxes) > 0:
        for box, cls, conf in zip(res_fe.boxes.xyxy, res_fe.boxes.cls, res_fe.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(cls.item())
            label = f"FE:{FE_NAMES[cls_idx]} {conf.item():.2f}"

            # 더 두꺼운 빨간색 박스
            cv2.rectangle(
                vis,
                (x1, y1),
                (x2, y2),
                color=(255, 0, 0),   # BGR
                thickness=4
            )
            # 라벨 텍스트 (박스 아래쪽에)
            cv2.putText(
                vis,
                label,
                (x1, min(y2 + 20, vis.shape[0] - 5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
                cv2.LINE_AA
            )

    # -------- 4) 시각화 결과 저장 + 필요 시 화면에도 표시 --------
    out_name = os.path.basename(img_path)
    out_path = os.path.join(OUT_OC_FE_DIR, out_name)

    # vis는 RGB라 BGR로 변환 후 저장
    vis_bgr = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, vis_bgr)
    print(f"[INFO] Saved vis image: {out_path}")

    # 필요하면 바로 plt로 보여주기
    # plt.figure(figsize=(6, 6))
    # plt.title(os.path.basename(img_path))
    # plt.imshow(vis)
    # plt.axis('off')
    # plt.show()

[INFO] Found 216 images.

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_missing1_new_002\frame_000000.jpg: 384x640 (no detections), 121.5ms
Speed: 1.8ms preprocess, 121.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_missing1_new_002\frame_000000.jpg: 384x640 (no detections), 70.7ms
Speed: 1.2ms preprocess, 70.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Saved vis image: demo\yolo\frame_000000.jpg

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_missing1_new_002\frame_000001.jpg: 384x640 (no detections), 78.7ms
Speed: 1.8ms preprocess, 78.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 g:\GitProjects\sessac_project\test_data\test_video\video_missing1_new_002\frame_000001.jpg: 384x640 (no detections), 84.3ms
Speed: 2.2ms preprocess, 84.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384,

In [12]:
# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"video_normal_new_002.mp4"
OUT_OC_FE_DIR1 = r"demo\yolo\video_normal_new_002"
images_to_video(OUT_OC_FE_DIR1, OUT_YOLO_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"video_normal_new_003.mp4"
OUT_OC_FE_DIR2 = r"demo\yolo\video_normal_new_003"
images_to_video(OUT_OC_FE_DIR2, OUT_YOLO_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_YOLO_VIDEO = r"video_missing1_new_002.mp4"
OUT_OC_FE_DIR3 = r"demo\yolo\video_missing1_new_002"
images_to_video(OUT_OC_FE_DIR3, OUT_YOLO_VIDEO, fps=7.5)

[INFO] Video saved: video_normal_new_002.mp4
[INFO] Video saved: video_normal_new_003.mp4
[INFO] Video saved: video_missing1_new_002.mp4


## mediapipe 시각화

In [15]:
import mediapipe as mp

mp_hands = mp.solutions.hands
HAND_CONNECTIONS = mp_hands.HAND_CONNECTIONS  # (index1, index2) 쌍들의 set

In [16]:
FRAMES_DIR1      = r"test_data\test_video\video_normal_new_002(fast)"      # 원본 프레임 폴더
LANDMARK_NPZ1    = r"test_data\test_in_model\test_npz\hands_video_normal_new_002.npz"  # 미리 저장했던 npz

FRAMES_DIR2      = r"test_data\test_video\video_normal_new_003"      # 원본 프레임 폴더
LANDMARK_NPZ2    = r"test_data\test_in_model\test_npz\hands_video_normal_new_003.npz"  # 미리 저장했던 npz

FRAMES_DIR3      = r"test_data\test_video\video_missing1_new_002"      # 원본 프레임 폴더
LANDMARK_NPZ3    = r"test_data\test_in_model\test_npz\hands_video_missing1_new_002.npz"  # 미리 저장했던 npz

OUT_MP_DIR      = r"demo\media"             # 랜드마크 시각화 결과 저장 폴더
os.makedirs(OUT_MP_DIR, exist_ok=True)


In [20]:
# ===== 프레임 로딩 =====
frame_paths = []
for ext in ("*.jpg", "*.png"):
    frame_paths.extend(glob.glob(os.path.join(FRAMES_DIR1, ext)))
frame_paths = natsorted(frame_paths)

print(f"[INFO] Found {len(frame_paths)} frames")

# ===== npz 로딩 =====
data = np.load(LANDMARK_NPZ1)
flat = data["hand_kps"]              # (235, 126)
T, D = flat.shape
print("[INFO] hand_kps shape:", flat.shape)

# ===== reshape: (T, 42, 3) =====
assert D % 3 == 0, "차원이 3으로 나누어 떨어지지 않음!"
num_points = D // 3                  # 42
landmarks = flat.reshape(T, num_points, 3)
print("[INFO] reshaped:", landmarks.shape)

# ===== frame / landmark 길이 맞추기 =====
T2 = min(T, len(frame_paths))
print(f"[INFO] Using {T2} frames")

# ===== 왼손/오른손 분리 =====
NUM_HAND_POINTS = 21
LEFT_IDX  = slice(0, 21)             # 0~20
RIGHT_IDX = slice(21, 42)            # 21~41

# ===== 시각화 =====
for i in range(T2):
    img_path = frame_paths[i]
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    h, w = img.shape[:2]
    lm = landmarks[i]                # (42, 3)

    # ----- 왼손 / 오른손 -----
    left  = lm[LEFT_IDX]             # (21, 3)
    right = lm[RIGHT_IDX]            # (21, 3)

    # ===== 포인트 그리기 함수 =====
    def draw_hand(points, color):
        # x,y 정규화 → 픽셀 변환
        xs = (points[:, 0] * w).astype(int)
        ys = (points[:, 1] * h).astype(int)

        # 점 그리기
        for x, y in zip(xs, ys):
            cv2.circle(img, (x, y), 3, color, -1)

        # 선 연결
        for c in HAND_CONNECTIONS:
            s, e = c
            x1, y1 = xs[s], ys[s]
            x2, y2 = xs[e], ys[e]
            cv2.line(img, (x1, y1), (x2, y2), color, 2)

    # ===== 실제 렌더링 =====

    FIXED_COLOR = (0, 255, 0)  # BGR, 연두색

    draw_hand(left,  FIXED_COLOR)
    draw_hand(right, FIXED_COLOR)
    
    # ===== 저장 =====
    out = os.path.join(OUT_MP_DIR, os.path.basename(img_path))
    cv2.imwrite(out, img)

print("[INFO] Mediapipe landmark visualization done.")


[INFO] Found 235 frames
[INFO] hand_kps shape: (235, 126)
[INFO] reshaped: (235, 42, 3)
[INFO] Using 235 frames
[INFO] Mediapipe landmark visualization done.


In [21]:
# ===== 프레임 로딩 =====
frame_paths = []
for ext in ("*.jpg", "*.png"):
    frame_paths.extend(glob.glob(os.path.join(FRAMES_DIR2, ext)))
frame_paths = natsorted(frame_paths)

print(f"[INFO] Found {len(frame_paths)} frames")

# ===== npz 로딩 =====
data = np.load(LANDMARK_NPZ2)
flat = data["hand_kps"]              # (235, 126)
T, D = flat.shape
print("[INFO] hand_kps shape:", flat.shape)

# ===== reshape: (T, 42, 3) =====
assert D % 3 == 0, "차원이 3으로 나누어 떨어지지 않음!"
num_points = D // 3                  # 42
landmarks = flat.reshape(T, num_points, 3)
print("[INFO] reshaped:", landmarks.shape)

# ===== frame / landmark 길이 맞추기 =====
T2 = min(T, len(frame_paths))
print(f"[INFO] Using {T2} frames")

# ===== 왼손/오른손 분리 =====
NUM_HAND_POINTS = 21
LEFT_IDX  = slice(0, 21)             # 0~20
RIGHT_IDX = slice(21, 42)            # 21~41

# ===== 시각화 =====
for i in range(T2):
    img_path = frame_paths[i]
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    h, w = img.shape[:2]
    lm = landmarks[i]                # (42, 3)

    # ----- 왼손 / 오른손 -----
    left  = lm[LEFT_IDX]             # (21, 3)
    right = lm[RIGHT_IDX]            # (21, 3)

    # ===== 포인트 그리기 함수 =====
    def draw_hand(points, color):
        # x,y 정규화 → 픽셀 변환
        xs = (points[:, 0] * w).astype(int)
        ys = (points[:, 1] * h).astype(int)

        # 점 그리기
        for x, y in zip(xs, ys):
            cv2.circle(img, (x, y), 3, color, -1)

        # 선 연결
        for c in HAND_CONNECTIONS:
            s, e = c
            x1, y1 = xs[s], ys[s]
            x2, y2 = xs[e], ys[e]
            cv2.line(img, (x1, y1), (x2, y2), color, 2)

    # ===== 실제 렌더링 =====

    FIXED_COLOR = (0, 255, 0)  # BGR, 연두색

    draw_hand(left,  FIXED_COLOR)
    draw_hand(right, FIXED_COLOR)
    
    # ===== 저장 =====
    out = os.path.join(OUT_MP_DIR, os.path.basename(img_path))
    cv2.imwrite(out, img)

print("[INFO] Mediapipe landmark visualization done.")


[INFO] Found 452 frames
[INFO] hand_kps shape: (452, 126)
[INFO] reshaped: (452, 42, 3)
[INFO] Using 452 frames
[INFO] Mediapipe landmark visualization done.


In [22]:
# ===== 프레임 로딩 =====
frame_paths = []
for ext in ("*.jpg", "*.png"):
    frame_paths.extend(glob.glob(os.path.join(FRAMES_DIR3, ext)))
frame_paths = natsorted(frame_paths)

print(f"[INFO] Found {len(frame_paths)} frames")

# ===== npz 로딩 =====
data = np.load(LANDMARK_NPZ3)
flat = data["hand_kps"]              # (235, 126)
T, D = flat.shape
print("[INFO] hand_kps shape:", flat.shape)

# ===== reshape: (T, 42, 3) =====
assert D % 3 == 0, "차원이 3으로 나누어 떨어지지 않음!"
num_points = D // 3                  # 42
landmarks = flat.reshape(T, num_points, 3)
print("[INFO] reshaped:", landmarks.shape)

# ===== frame / landmark 길이 맞추기 =====
T2 = min(T, len(frame_paths))
print(f"[INFO] Using {T2} frames")

# ===== 왼손/오른손 분리 =====
NUM_HAND_POINTS = 21
LEFT_IDX  = slice(0, 21)             # 0~20
RIGHT_IDX = slice(21, 42)            # 21~41

# ===== 시각화 =====
for i in range(T2):
    img_path = frame_paths[i]
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    h, w = img.shape[:2]
    lm = landmarks[i]                # (42, 3)

    # ----- 왼손 / 오른손 -----
    left  = lm[LEFT_IDX]             # (21, 3)
    right = lm[RIGHT_IDX]            # (21, 3)

    # ===== 포인트 그리기 함수 =====
    def draw_hand(points, color):
        # x,y 정규화 → 픽셀 변환
        xs = (points[:, 0] * w).astype(int)
        ys = (points[:, 1] * h).astype(int)

        # 점 그리기
        for x, y in zip(xs, ys):
            cv2.circle(img, (x, y), 3, color, -1)

        # 선 연결
        for c in HAND_CONNECTIONS:
            s, e = c
            x1, y1 = xs[s], ys[s]
            x2, y2 = xs[e], ys[e]
            cv2.line(img, (x1, y1), (x2, y2), color, 2)

    # ===== 실제 렌더링 =====

    FIXED_COLOR = (0, 255, 0)  # BGR, 연두색

    draw_hand(left,  FIXED_COLOR)
    draw_hand(right, FIXED_COLOR)
    
    # ===== 저장 =====
    out = os.path.join(OUT_MP_DIR, os.path.basename(img_path))
    cv2.imwrite(out, img)

print("[INFO] Mediapipe landmark visualization done.")


[INFO] Found 216 frames
[INFO] hand_kps shape: (216, 126)
[INFO] reshaped: (216, 42, 3)
[INFO] Using 216 frames
[INFO] Mediapipe landmark visualization done.


In [25]:
# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_MEDIA_VIDEO = r"demo\media\hands_video_normal_new_002.mp4"
OUT_OC_FE_DIR1 = r"demo\media\hands_video_normal_new_002"
images_to_video(OUT_OC_FE_DIR1, OUT_MEDIA_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_MEDIA_VIDEO = r"demo\media\hands_video_normal_new_003.mp4"
OUT_OC_FE_DIR2 = r"demo\media\hands_video_normal_new_003"
images_to_video(OUT_OC_FE_DIR2, OUT_MEDIA_VIDEO, fps=7.5)

# YOLO 시각화 이미지를 7.5fps 영상으로 변환
OUT_MEDIA_VIDEO = r"demo\media\hands_video_missing1_new_002.mp4"
OUT_OC_FE_DIR3 = r"demo\media\hands_video_missing1_new_002"
images_to_video(OUT_OC_FE_DIR3, OUT_MEDIA_VIDEO, fps=7.5)

[INFO] Video saved: demo\media\hands_video_normal_new_002.mp4
[INFO] Video saved: demo\media\hands_video_normal_new_003.mp4
[INFO] Video saved: demo\media\hands_video_missing1_new_002.mp4
